In [578]:
### 필요한 라이브러리 불러오기
import OpenDartReader
import numpy as np    
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


api_key = '본인의 인증키 입력'
dart = OpenDartReader(api_key) 

def finance(company, From, To) : 

    ### Dictionary
    df_info = {}
     
    for year in range(From, To+1) : 

        ### 재무제표
        fs = dart.finstate(company, year, reprt_code='11011') # 재무제표(전체)
        """
        reprt_code : '11013' = 1분기보고서, '11012' = 반기보고서, '11014' = 3분기보고서, '11011' = 사업보고서
        """
        fs_is = fs[fs['sj_div'].str.contains('IS')] # 재무제표(전체) > 손익계산서
        fs_bs = fs[fs['sj_div'].str.contains('BS')] # 재무제표(전체) > 재무제표
        fs_equity = fs_bs[fs_bs['account_nm'].str.contains('자본총계')] # 재무제표(전체) > 재무제표 > 자본총계
        fs_assets = fs_bs[fs_bs['account_nm'].str.contains('자산총계')] # 재무제표(전체) > 재무제표 > 자산총계
        fs_revenue = fs_is[fs_is['account_nm'].str.contains('매출액')] # 재무제표(전체) > 손익계산서 > 매출액
        fs_income = fs_is[fs_is['account_nm'].str.contains('영업이익')] # 재무제표(전체) > 손익계산서 > 영업이익
        fs_profit = fs_is[fs_is['account_nm'].str.contains('당기순이익')] # 재무제표(전체) > 손익계산서 > 당기순이익

        ### 재무제표 전체(재무상태표, 손익계산서, 자본변동표, 현금흐름표 등)
        fs_all = dart.finstate_all(company, year) # 재무제표(전체)
        fs_cf = fs_all[fs_all['sj_div'].str.contains('CF')] # 재무제표(전체) > 현금흐름표
        fs_OCF = fs_cf[fs_cf['account_id'].str.contains('CashFlowsFromUsedInOperatingActivities')] # 재무제표(전체) > 현금흐름표 > 영업활동현금흐름
        fs_aucqusition = fs_cf[fs_cf['account_nm'].str.contains('취득')] # 재무제표(전체) > 현금흐름표 >  '취득'
        fs_CAPEX_P = fs_aucqusition[fs_aucqusition['account_nm'].str.contains('유형자산')] # 재무제표(전체) > 현금흐름표 >  유형자산의취득
        fs_CAPEX_I = fs_aucqusition[fs_aucqusition['account_nm'].str.contains('무형자산')] # 재무제표(전체) > 현금흐름표 >  무형자산의취득

        ### 배당에 관한 사항
        dv = dart.report(company, '배당', year) # 배당에 관한 사항
        dv_dps = dv[ dv['se'] == '주당 현금배당금(원)' ] # 배당에 관한 사항 > 주당 배당금
        dv_eps = dv[ dv['se'].str.contains('주당순이익') ] # 배당에 관한 사항 > 주당 순이익
        dv_yield = dv[ dv['se'].str.contains('현금배당수익률') ] # 배당에 관한 사항 > 배당수익률
        dv_TD = dv[ dv['se'].str.contains('현금배당금총액') ] # 배당에 관한 사항 > 배당금 총액

   
        equity_this = int(fs_equity[['thstrm_amount']].iloc[0,0].replace(',', '').strip()) # 당해년도 자본총계
        equity_last = int(fs_equity[['frmtrm_amount']].iloc[0,0].replace(',', '').strip()) # 직전년도 자본총계
        equity = (equity_this + equity_last ) / 2 # 평균자본총계 
        assets_this = int(fs_assets[['thstrm_amount']].iloc[0,0].replace(',', '').strip()) # 당해년도 자본총계
        assets_last = int(fs_assets[['frmtrm_amount']].iloc[0,0].replace(',', '').strip()) # 직전년도 자본총계
        assets = (assets_this + assets_last ) / 2 # 평균자산총계
        revenue = int(fs_revenue[['thstrm_amount']].iloc[0,0].replace(',', '')) # 매출액
        income = int(fs_income[['thstrm_amount']].iloc[0,0].replace(',', '')) # 영업이익
        profit = int(fs_profit[['thstrm_amount']].iloc[0,0].replace(',', '')) # 순이익
        DPS = int( dv_dps[ ['thstrm'] ].iloc[0, 0].replace(',', '').strip() ) # 주당 배당금
        EPS = int( dv_eps[ ['thstrm'] ].iloc[0, 0].replace(',', '').strip() ) # 주당 순이익
        Yield = int( dv_yield[ ['thstrm'] ].iloc[0, 0].replace('.', '').strip() ) / 10000 # 배당수익률
        TD = int( dv_TD[ ['thstrm'] ].iloc[0, 0].replace(',', '').strip() ) # 배당금 총액
        OCF = int(fs_OCF[['thstrm_amount']].iloc[0,0]) # 당해년도
        CAPEX_P = int(fs_CAPEX_P[['thstrm_amount']].iloc[0,0]) # 유형자산의취득
        CAPEX_I = int(fs_CAPEX_I[['thstrm_amount']].iloc[0,0]) #  무형자산의취득
        CAPEX = CAPEX_P + CAPEX_I
        ROE = profit / equity # 자기자본수익률
        ROA = profit / assets # 자산수익률
        FCF = OCF - CAPEX # 잉여현금흐름
        OPM = income / revenue # 영업이익률
        NPM = profit / revenue # 순이익률
        PR = DPS / EPS # 배당성향
        
        ### dictionary에 담기
        df_info[str(year)] = \
        f'{revenue:,}'[:-8], \
        f'{income:,}'[:-8], \
        f'{profit:,}'[:-8], \
        f'{OCF:,}'[:-8], \
        f'{FCF:,}'[:-8], \
        f'{TD:,}', \
        f'{OPM:.1%}', \
        f'{NPM:.1%}', \
        f'{ROE:.1%}', \
        f'{ROA:.1%}', \
        f'{EPS:,}', \
        f'{DPS:,}', \
        f'{PR:.1%}', \
        f'{Yield:.1%}', \
        # f'{value:,}'[:-8] : ###,, / f'{value:.1%}' : #.#%
        
    ### 출력 ###
    df = pd.DataFrame(data = df_info, \
                      index = ['Revenue', 'Operating Profit', 'Net Profit', 
                               'Operating Cash Flow', 'Free Cash Flow', 'Total Dividend',
                               'Operating Profit Margin', 'Net Profit Margin', 
                               'Return on Equity', 'Return on Assets', 
                              'EPS', 'DPS', 'Payout Ratio', 'Yield'])

    company_nm = dart.company(company)['stock_name'] # 종목명
    company_nm_eng = dart.company(company)['corp_name_eng'] # 회사 영문명
    company_stock_code = dart.company(company)['stock_code'] # 종목코드    
    print(company_nm, company_nm_eng, company_stock_code, '(단위 : %, 백만원, 원)')
    return df   

In [579]:
finance('케이티앤지', 2015, 2019)

케이티앤지 KT&G Corporation 033780 (단위 : %, 백만원, 원)


,2015,2016,2017,2018,2019
Revenue,"4,169,839","4,503,279","4,667,193","4,471,515","4,963,202"
Operating Profit,"1,365,904","1,470,101","1,426,114","1,255,106","1,382,018"
Net Profit,"1,032,219","1,226,029","1,164,208","898,659","1,038,670"
Operating Cash Flow,"1,259,235","1,497,761","1,139,868","822,057","1,042,489"
Free Cash Flow,"1,042,912","1,326,794","815,145","421,894","785,293"
Total Dividend,"428,284","454,554","505,061","505,061","556,952"
Operating Profit Margin,32.8%,32.6%,30.6%,28.1%,27.8%
Net Profit Margin,24.8%,27.2%,24.9%,20.1%,20.9%
Return on Equity,17.2%,18.3%,15.6%,11.3%,12.3%
Return on Assets,12.8%,13.3%,11.8%,8.9%,10.0%
